In [ ]:
import xarray as xr
import os

ipath = '/glade/work/gangrade/R18_HUC12_summa/forcings/'
opath = '/glade/work/nijssen/scratch'

id_list_file = '/glade/work/andywood/wreg/gis/shapes/cali_huc12/cali_hrus.txt'
ifile = os.path.join(ipath, 'R18_HUC12.forcing.198001.nc')
ofile = os.path.join(opath, 'R18_HUC12.forcing.198001.nc'.replace('.nc', '_subset.nc'))

# dimensions along which to subset
# note the assumption / limitation is that variables
# are not expected to co-vary along multiple dimensions

subset_dims = ['hru']
subset_files = [id_list_file]

ds = xr.open_dataset(ifile)

# determine which variables do not vary along any of the subsetted dimensions
vars_without = []
for var in ds.variables:
    if not set(subset_dims).intersection(ds[var].dims):
        vars_without.append(var)

ds_subset = ds[vars_without]

for subset_dim, id_file in zip(subset_dims, subset_files):

    with open(id_file) as f:
        ids = [int(x) for x in f]

    # take the other variables
    vars_with = list(set(ds.variables).difference(set(vars_without)))

    ds_with = ds[vars_with]
    ds_with = ds_with.where(ds_with.hruId.isin(ids), drop=True)
    ds_subset = ds_subset.merge(ds_with)

# make sure that the subsetted types are the same as the original ones
for var in ds_subset.variables:
    ds_subset[var] = ds_subset[var].astype(ds[var].dtype)

ds_subset.to_netcdf(ofile)

a = set(ds_subset.hruId.values)

set(ids).difference(a)